In [294]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Класс, который в последствии добавляется в словарь для удобного выбора
class Node:
    def __init__(
        self,
        feature=None,
        threshold=None,
        childs=None,
        # left = None,
        # right = None,
        value=None,
        proba_value = None
    ):
        self.feature = feature
        self.threshold = threshold
        self.childs = childs
        self.value = value
        # self.right = right
        # self.left = left
        self.proba_value = proba_value

    def is_leaf_node(self):
        return self.value is not None
    


class DecisionTree:
    def __init__(self, max_depth=10, min_samples=10):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.tree = None
        self.classes = None


    def fit(self, X, y):
        self.classes = np.unique(y)
        self.tree = self.grow_tree(X, y)

    def calck_unic(self, a: list):
        keys = np.unique(a)
        # return {key: a[a == key].shape[0] for key in keys}
        return (a[a == key].shape[0] for key in keys), keys

    def predict(self, X):
        return np.array([self.travers_tree(x, self.tree) for x in X])

    def predict_proba(self, X):
        return np.array([self.travers_proba_tree(x, self.tree) for x in X])

    def entropy(self, y: np.ndarray):
        hist, _ = self.calck_unic(y)
        n = y.shape[0]
        # hist = {key: val / len(y) for key, val in hist.items()}
        info = -np.sum(np.fromiter((val/n * np.log2(val/n) for val in hist), dtype=np.float64))

        return info
    
    #======================================
    def gini(self, y:np.ndarray):
        uitems = self.calck_unic(y)
        n = y.shape[0]
        return 1 - np.sum(np.fromiter(((val/n)**2 for val in uitems), dtype=np.float64))
    #======================================

    def information_gain(self, X_column: list, y: list):
        if np.unique(y).shape[0] == 1:
            return 0

        n = y.shape[0]
        parent = self.entropy(y)
        # print("point1")
        uitems, keys = self.calck_unic(X_column)
        uitems = [i for i in uitems]
        # print("point2")
        #======================================
        # left_inds = np.argwhere(X_column <= threshold).flatten()
        # right_inds = np.argwhere(X_column > threshold).flatten()
        # # print(left_inds)
        # # print(X_column.shape)
        # # print(y.shape)
        # gini_left = self.gini(y[left_inds])
        # gini_right = self.gini(y[right_inds])
        # gini_split = (len(left_inds) / n) * gini_left +  (len(right_inds) / n) * gini_right
        # return gini_split, threshold
        #====================================== 
        # print(X_column.shape, y.shape )
        info_x = np.sum(
            [uitems[i] / n * self.entropy(y[X_column == keys[i]]) for i in range(len(uitems))]
        )
        # print("point3")
        split_info = -np.sum(
            [val / n * np.log2(val / n) for val in uitems if val > 0]
        )
        # print("point4")
        if split_info != 0:
            return (parent - info_x) / split_info, keys
        else:
            return 0, keys

    def most_common(self, y):
        labels = np.unique(y)
        vals, _ = self.calck_unic(y)
        return labels[np.argmax(vals)]

    def proba_val(self, y):
        n = y.shape[0]
        probas = np.zeros(self.classes.shape[0])
        vals, keys = self.calck_unic(y)

        probas[np.in1d(self.classes, keys)] = np.fromiter(vals, dtype=np.float64) / n
        return probas

    def best_split(self, X, y):
        best_feature = None
        best_gain = -1
        # best_gini = 1000000
        # uitems = []
        for i in range(X.shape[1]):
            #==============================================
            # trasholds = np.random.choice(np.unique(X[:, i]), 10)
            # print("new_i")
            # for trashold in trasholds:
            #==============================================
            # print("test1")
            gain, now_uitems = self.information_gain(X[:, i], y)
            # print("test2")
            # print(gain)
            if gain > best_gain:
                # print("yep")
                best_gain = gain
                best_feature = i
                uitems = now_uitems

        return best_feature, uitems

    def grow_tree(self, X, y, depth=0):
        n_samples = X.shape[0]
        n_labels = np.unique(y).shape[0]

        if n_samples <= self.min_samples or depth >= self.max_depth or n_labels <= 1:
            return Node(value=self.most_common(y), proba_value=self.proba_val(y))

        best_feature, ukeys = self.best_split(X, y)
        #==============================================
        # best_feature, best_threshold = self.best_split(X, y)
        # l_inds = np.argwhere(X[:, best_feature] <= best_threshold).flatten()
        # r_inds = np.argwhere(X[:, best_feature] > best_threshold).flatten()
        # # print("depth2:", depth)
        # if len(l_inds) == 0 or len(r_inds) == 0:
        #     return Node(value=self.most_common(y), proba_valUe=self.proba_val(y))
        
        # # print("depth3:", depth)
        # left = self.grow_tree(X = X[l_inds], y = y[l_inds], depth = depth+1)
        # right = self.grow_tree(X = X[r_inds], y = y[r_inds], depth = depth+1)

        # return Node(best_feature, best_threshold, left, right)
        #==============================================
        # В словаре содержатся не словари, а Node По сути, словарь содержит ссылки на объекты, а нужен он для более удобной навигации.
        childs = {
            key: self.grow_tree(
                X[X[:, best_feature] == key],
                y[X[:, best_feature] == key],
                depth=depth + 1,
            )
            for key in ukeys
        }

        return Node(best_feature, childs=childs)

    #==============================================
    # def travers_tree(self, x, tree):
    #     if tree.is_leaf_node():
    #         return tree.value
        
    #     return self.travers_tree(x, tree.left) if tree.threshold >= x[tree.feature] else self.travers_tree(x, tree.right)

    # def travers_proba_tree(self, x, tree):
    #     if tree.is_leaf_node():
    #         # print(tree.proba_value)
    #         return tree.proba_value
        
    #     return self.travers_proba_tree(x, tree.left) if tree.threshold >= x[tree.feature] else self.travers_proba_tree(x, tree.right)
    #==============================================

    def travers_tree(self, x, tree):
        if tree.is_leaf_node():
            return tree.value

        return self.travers_tree(
            x,
            tree.childs.get(
                x[tree.feature], tree.childs.get(list(tree.childs.keys())[0])
            ),
        )

    def travers_proba_tree(self, x, tree):
        if tree.is_leaf_node():
            return tree.proba_value

        return self.travers_proba_tree(
            x,
            tree.childs.get(
                x[tree.feature], tree.childs.get(list(tree.childs.keys())[0])
            ),
        )


In [43]:
import pandas as pd
import matplotlib.pyplot as plt

# Загрузка данных
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = pd.read_csv(url, names=names)
transform = {'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2}
dataset["class"] = dataset["class"].apply(lambda x: transform[x])

In [253]:
data = pd.read_csv("./Data/X_train.csv")
data.drop(labels="measurement_number", axis=1 , inplace=True)
data = data.groupby("series_id").agg(['mean', 'std', 'median'])
data.columns = [f'{col}_{stat}' for col, stat in data.columns]
data = data.reset_index()
data.drop(labels=["series_id","row_id_mean","row_id_std", "row_id_median"], axis=1 , inplace=True)
data = data.round(2).to_numpy(dtype=np.float64)
y = pd.read_csv("./Data/y_train.csv")["group_id"].to_numpy(dtype=np.int64)
# X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2)
# print(np.unique(y_test))
# data.head(40)

In [254]:
# X = dataset[dataset.columns[:-1]].to_numpy()[:]
# y = dataset["class"].to_numpy()

# x_train, x_test, y_train1, y_test1 = train_test_split(X, y, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2)
clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train.flatten())
otv = clf.predict(X_test)
print(otv)
# print(y_test.flatten()-1)
# print(X_test)
print(f1_score(y_test.flatten(), otv, average="micro"))

[18 45  3 37 19 51 10 14  1 59 67 41 22 40 59 19 11 69 41 38 17 54 31 68
 51 70 23 33 39 16 22 69 33 44 53  3 51 41 15 42 17 16 22 32  4  8 44 61
 69 18 60 38 70 44 46 49 64 55 53 18 12  1 68 20  3 60 39 22 51  1 20 33
 61  8 15 46 11 23 46 16  1 68 16 46 46 22 17 13 47 33 54 18  0  4 29 13
 28  8 39 10 64 46 72 44 15  3 54 11 39 46  4 69 33 46  3 38 53 22 22 13
  4 12 18 14  8  4 29 45 40 66 33 39  1  0  2 38 53 34  7 14 49 32  1 49
 15 19 60 67 43 36 30 13  8 12  2 18 39  4 59 27 38 35  4 60 35 49 25 20
 25 14 42 70  7 42 39 35 20 22 14 25 33  6 17 10 13 15 46 11 71 60 16 10
 31 21 15 48  4 33  1 18  1 44 40 41 70 44 38  2  4 59 18 20 35  9 55 22
 48 36 30 46 18 46 22 39 65 52 13 10 51 17 17 48 52 29 14 22 14 48 15  3
 18 33 15 65 20 45 22 14 43  8  1 40 36  6 22 43 25  1  6 16 53 43 15  8
 69 60 31 37 60 46 22 62  4 44 66 46 42 68 22 45 65  9 23 71 38  8 18  7
 65 51 36 32 34 19  6 18 54 23 40 53 37 64 25 35 69 57 18 62  2 69 60 13
 16  0 55  4 66 66 15 61 42 60 14 18 64 65 53 15 15

In [338]:
class RandomForest:
    def __init__(self, n_trees = 10, max_deep = 10, min_samples = 10):
        self.trees = [0 for i in range(n_trees)]
        self.tree_inds = [0 for i in range(n_trees)]
        self.n_trees = n_trees
        self.max_deep = max_deep
        self.min_samples = min_samples
        self.num_classes=None

    def fit(self, X, y):
        n_features = int(X.shape[1]**0.5+0.5)
        # n_features = X.shape[0]
        self.num_classes = np.unique(y).shape[0]

        for i in range(self.n_trees):
            clf = DecisionTree(max_depth=self.max_deep, min_samples=self.min_samples)
            inds =  np.random.choice(np.arange(X.shape[1]), n_features)
            # inds =  np.random.choice(np.arange(X.shape[0]), n_features, replace=True)
            clf.fit(X[:, inds], y)
            # print(y)
            # clf.fit(X, y)
            # print(clf.predict(x_test[:,inds ]))
            self.trees[i] = clf
            self.tree_inds[i] = inds
    
    def predict(self, X):
        otv = np.zeros((X.shape[0], self.num_classes))
        
        for i, clf in enumerate(self.trees):
            otv[np.arange(X.shape[0]), clf.predict(X[:, self.tree_inds[i]])] += 1
            # otv[np.arange(X.shape[0]), clf.predict(X)] += 1
            # print(otv)
        # print(otv)
        return np.argmax(otv, axis=1)
    
    def predict_proba(self, X):
        otv = np.zeros((X.shape[0], self.num_classes))
        for i, clf in enumerate(self.trees):
            otv += clf.predict_proba(X[:, self.tree_inds[i]])

        return otv


In [342]:
clf = RandomForest(n_trees=800, max_deep=100)
clf.fit(X_train, y_train.flatten())
# otv = clf.predict(X_test)
otv = np.argmax(clf.predict_proba(X_test), axis=1)
print(otv)
# print(y_test.flatten())
print(f1_score(y_test.flatten() , otv, average="micro"))

[18 16  3 37 15 61 49 14 11 59 67 41 32 18 59 72 11 56 41 25 57 48 31 68
 51 70 23 60 39 16  0 69 33 21 70  3 61 41 25 72 57 16 22 20  2 60 36 22
 69 26 71 59 70 44 26 51 64 66 64 40 12  9 22 45 26 71 39 22 61  1 62 33
 51  8 55  3 11 23  3 49 47 68 16 61 61  0  1 30 47 33 48 72 68 47 29 13
 35 28 72 52 64 26 72 47 72  3 48 11 40 51  4 56 33  3  3 40 15  0  4 13
  4 12 42 15  8  4 29 45 45 71 60 70 29 68 46 27 53 34  7 14 43 61 11 13
 54 19 60 67 42 36 34 13  8 12 65 18 63 41 59 13 42 69  4 71 69 68 25  0
 25 64 42 70 10 72 70 27  7  7 14 25 33 29 57  7 12 43 26 11 66 66 16 10
 31 21 55 48  4 60  9 18 56 59 69 41 70 36 45 44  4 59  9 68 35 27 71  0
 48 36 34 26 42 27 22 55 65 14 34 31 51 21 57 15 52 41 14 22 14 48 23  3
 38 33 23 61 20 22  0 52 70 49  9 59 36  9 22 43 25 43  6 16 53 62 49  8
 69 71 22 69 45  3  7 62  9 59 66 26 42 68 32 46 50 21 23 71 43 38 42 22
 20 68 36 32 34 19  6 42 14 23 40 53 69 64 25 69 69 57 18 62  2 69 71 13
 62 68 44 44 66 66 15 22 70 66 14 18 64 61 54 25 67